In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import geopy.distance
from matplotlib.ticker import FuncFormatter

# 파일 경로와 범위 설정
base_path_2024 = "/content/drive/MyDrive/여의도_불꽃축제_데이터/인구종합/"
file_template_2024 = "2024-10-05_{:02d}_인구종합.csv"  # 2024년 데이터 템플릿
output_path = "/content/drive/MyDrive/여의도_불꽃축제_데이터/지도결과/"

# 출력 폴더가 없으면 생성
os.makedirs(output_path, exist_ok=True)

# 중심 지역의 좌표 설정 (위도, 경도)
center_lat = 37.52663
center_lon = 126.9338

# 한산한 지역의 좌표 설정 (위도, 경도)
quiet_area_lat = 37.519590
quiet_area_lon = 126.926318

# 거리 제한 (200x200 미터, 위도/경도 단위)
distance_limit = 0.0018  # 약 200미터

# 인구수 비교 그래프 생성 함수
def generate_population_comparison_graph(file_template, date_label, area_label):
    # 시간별 인구 수 데이터를 저장할 리스트
    time_population_entire_area = []
    time_population_center_area = []
    time_population_quiet_area = []

    # 각 시간대 (00시부터 23시까지)
    for hour in range(24):
        file_path = base_path_2024 + file_template.format(hour)
        try:
            # CSV 파일 읽기
            df = pd.read_csv(file_path)

            # 전체 지역 인구 수 계산
            total_population_entire = df['POPL_CNT'].sum()

            # 중심 지역과 한산한 지역의 인구 수 계산
            filtered_data_center = []
            filtered_data_quiet = []

            # 각 좌표의 인구 수 계산
            for _, row in df.iterrows():
                x, y, popl = row['X_COORD'], row['Y_COORD'], row['POPL_CNT']

                if not pd.isna(x) and not pd.isna(y):
                    # 중심 지역과의 거리 계산
                    dist_center = geopy.distance.distance((center_lat, center_lon), (y, x)).km
                    if dist_center <= 0.2:  # 중심 지역은 200미터 이내
                        filtered_data_center.append(popl)

                    # 한산한 지역과의 거리 계산
                    dist_quiet = geopy.distance.distance((quiet_area_lat, quiet_area_lon), (y, x)).km
                    if dist_quiet <= 0.2:  # 한산한 지역은 200미터 이내
                        filtered_data_quiet.append(popl)

            # 각 지역의 인구 수 합산
            total_population_center = sum(filtered_data_center)
            total_population_quiet = sum(filtered_data_quiet)

            # 시간별 인구 수 저장
            time_population_entire_area.append((hour, total_population_entire))
            time_population_center_area.append((hour, total_population_center))
            time_population_quiet_area.append((hour, total_population_quiet))

        except FileNotFoundError:
            print(f"파일을 찾을 수 없습니다: {file_path}")
            continue

    # 시간별 인구 수 데이터를 DataFrame으로 생성
    time_population_df_entire_area = pd.DataFrame(time_population_entire_area, columns=['Hour', 'Total Population Entire Area'])
    time_population_df_center_area = pd.DataFrame(time_population_center_area, columns=['Hour', 'Total Population Center Area'])
    time_population_df_quiet_area = pd.DataFrame(time_population_quiet_area, columns=['Hour', 'Total Population Quiet Area'])

    # 시간대별 인구 수 그래프 그리기 (정규화 없이 원본 인구 수)
    plt.figure(figsize=(10, 6))

    # 전체 지역, 중심 지역, 한산한 지역에 대한 인구 수 그래프
    plt.plot(time_population_df_entire_area['Hour'], time_population_df_entire_area['Total Population Entire Area'],
             marker='o', color='r', linestyle='-', linewidth=2, markersize=6, label="Entire Area (Raw Population)")
    plt.plot(time_population_df_center_area['Hour'], time_population_df_center_area['Total Population Center Area'],
             marker='x', color='b', linestyle='--', linewidth=2, markersize=6, label="Center Area (Raw Population)")
    plt.plot(time_population_df_quiet_area['Hour'], time_population_df_quiet_area['Total Population Quiet Area'],
             marker='s', color='g', linestyle='-.', linewidth=2, markersize=6, label="Quiet Area (Raw Population)")

    # 제목과 레이블 설정
    plt.title(f'Population Comparison by Hour ({date_label})', fontsize=14)
    plt.xlabel('Hour of the Day', fontsize=12)
    plt.ylabel('Total Population (people)', fontsize=12)
    plt.xticks(range(24))
    plt.grid(True)
    plt.tight_layout()

    # 범례 추가
    plt.legend(loc='upper left')

    # 그래프 파일로 저장
    graph_file_path = output_path + f"population_comparison_raw_by_hour_{area_label}.png"
    plt.savefig(graph_file_path)

    # 그래프 표시
    plt.show()

    # 저장된 그래프 경로 출력
    print(f"그래프가 저장되었습니다: {graph_file_path}")

# 2024-10-05 데이터에 대한 비교 그래프 생성
generate_population_comparison_graph(file_template_2024, "2024-10-05", "2024")